In [ ]:
# ============================================
# Cell 0: 安装依赖（只需要运行一次）
# ============================================
!pip -q install ultralytics opencv-python-headless pandas numpy plotly scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print("✅ 视频已上传：", video_path)


Saving 58168_003392_Sideline.mp4 to 58168_003392_Sideline.mp4
✅ 视频已上传： 58168_003392_Sideline.mp4


In [ ]:
import os

print(os.path.exists(out_path))
if os.path.exists(out_path):
    print("文件大小：", os.path.getsize(out_path), "bytes")

True
文件大小： 25607352 bytes


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from IPython.display import Video

# 1. 确认你已经有 video_path，比如前面用 files.upload() 得到
print("当前视频文件：", video_path)

# 2. 加载 YOLO 模型
model = YOLO("yolov8n.pt")   # 可以换成 yolov8s.pt 等

# 3. 读取原视频 FPS，用来写预览视频
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
if fps <= 0:
    fps = 25  # 兜底

out_path = "yolo_preview.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = None

print("开始生成 YOLO 预览视频...")

# 4. 一遍跑完整个视频，边检测边画框
for result in model.track(
    source=video_path,
    tracker="bytetrack.yaml",  # 使用 ByteTrack 跟踪
    stream=True,
    persist=True,
    verbose=False
):
    # 原始帧（BGR）
    frame = result.orig_img.copy()

    boxes = result.boxes
    ids = boxes.id
    cls = boxes.cls
    xyxy = boxes.xyxy

    if writer is None:
        h, w = frame.shape[:2]
        writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    if ids is not None:
        ids = ids.cpu().numpy().tolist()
    else:
        ids = [None] * len(xyxy)

    cls = cls.cpu().numpy().tolist()
    xyxy = xyxy.cpu().numpy().tolist()

    for box_id, box, cls_id in zip(ids, xyxy, cls):
        x1, y1, x2, y2 = map(int, box)

        # 区分人和球：YOLO COCO: 0=person, 32=sports ball
        cls_id = int(cls_id)
        if cls_id == 0:
            color = (0, 255, 0)   # 人：绿色
        elif cls_id == 32:
            color = (0, 0, 255)   # 球：红色
        else:
            color = (255, 255, 0) # 其他：黄色

        # 画框
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # 文本：ID + 类别
        label_id = f"id:{int(box_id)}" if box_id is not None else "id:-"
        label_cls = "person" if cls_id == 0 else ("ball" if cls_id == 32 else f"cls:{cls_id}")
        label = f"{label_id} {label_cls}"

        cv2.putText(
            frame,
            label,
            (x1, max(15, y1 - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            1,
            cv2.LINE_AA
        )

    writer.write(frame)

if writer is not None:
    writer.release()

frame_count = 0

for result in model.track(
    source=video_path,
    tracker="bytetrack.yaml",
    stream=True,
    persist=True,
    verbose=False
):
    frame_count += 1
    if frame_count % 50 == 0:
        print("已经处理帧数：", frame_count)

    frame = result.orig_img.copy()
    ...
    writer.write(frame)

print("循环结束，总共处理帧数：", frame_count)

print("✅ 预览视频已生成：", out_path)


当前视频文件： 58168_003392_Sideline.mp4
开始生成 YOLO 预览视频...
已经处理帧数： 50
已经处理帧数： 100
已经处理帧数： 150
已经处理帧数： 200
已经处理帧数： 250
已经处理帧数： 300
已经处理帧数： 350
已经处理帧数： 400
已经处理帧数： 450
已经处理帧数： 500
已经处理帧数： 550
已经处理帧数： 600
已经处理帧数： 650
已经处理帧数： 700
循环结束，总共处理帧数： 711
✅ 预览视频已生成： yolo_preview.mp4


In [ ]:
Video(out_path)

In [ ]:
from google.colab import files
files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh

total 43M
-rw-r--r-- 1 root root 3.9M Nov 14 04:33 '58168_003392_Sideline (1).mp4'
-rw-r--r-- 1 root root 3.9M Nov 14 04:37 '58168_003392_Sideline (2).mp4'
-rw-r--r-- 1 root root 3.9M Nov 14 04:03  58168_003392_Sideline.mp4
-rw-r--r-- 1 root root  119 Nov 14 04:33  homography.json
drwxr-xr-x 1 root root 4.0K Nov 12 14:30  sample_data
-rw-r--r-- 1 root root  35K Nov 14 04:04  tracking_with_homography.csv
-rw-r--r-- 1 root root 361K Nov 14 04:41  video_tracking.csv
-rw-r--r-- 1 root root  25M Nov 14 05:31  yolo_preview.mp4
-rw-r--r-- 1 root root 6.3M Nov 14 04:04  yolov8n.pt


In [ ]:
import pandas as pd
import plotly.express as px

# ========== 1) 读取你的视频 tracking CSV ==========
csv_path = "video_tracking.csv"   # 如果名字不同，在这里改
df = pd.read_csv(csv_path)

# 简单 sanity check
print(df.head())
print("Total rows:", len(df))
print("Unique frameId:", df["frameId"].nunique())
print("Unique playerId:", df["playerId"].nunique())

# ========== 2) 定义球场样式（和你之前 NFL 代码类似） ==========
def field_shapes_matplotlib_style():
    shapes = []
    # 草地
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="darkgreen", layer="below"))
    # 两端区
    shapes.append(dict(type="rect", x0=0,   x1=10,  y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(220,20,60,0.85)", layer="below"))
    shapes.append(dict(type="rect", x0=110, x1=120, y0=0, y1=53.3,
                       line=dict(width=0), fillcolor="rgba(255,69,0,0.85)", layer="below"))
    # 10 码整线
    for x in range(0, 121, 10):
        shapes.append(dict(type="line", x0=x, x1=x, y0=0, y1=53.3,
                           line=dict(color="white", width=2), layer="below"))
    # 四组短刻度
    short_ticks = [(0.4, 0.7), (52.5, 53.0), (22.91, 23.57), (29.73, 30.39)]
    for x in range(11, 110):
        for y0, y1 in short_ticks:
            shapes.append(dict(type="line", x0=x, x1=x, y0=y0, y1=y1,
                               line=dict(color="white", width=1), layer="below"))
    # 外框
    shapes.append(dict(type="rect", x0=0, x1=120, y0=0, y1=53.3,
                       line=dict(color="white", width=2),
                       fillcolor="rgba(0,0,0,0)", layer="below"))
    return shapes

def field_annotations_matplotlib_style():
    ann = []
    # 码数数字（双向）
    for x in range(20, 110, 10):
        number = x if x <= 50 else 120 - x
        ann.append(dict(x=x, y=5, text=str(number - 10), showarrow=False,
                        font=dict(size=20, color="white"), xanchor="center", yanchor="middle"))
        ann.append(dict(x=x - 0.95, y=53.3 - 5, text=str(number - 10), showarrow=False,
                        font=dict(size=20, color="white"), textangle=180,
                        xanchor="center", yanchor="middle"))
    # TD 标记
    ann.append(dict(
        x=5, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    ann.append(dict(
        x=115, y=26.65, text="TD", showarrow=False,
        font=dict(size=52, color="white", family="Arial Black"),
        textangle=-90, xanchor="center", yanchor="middle", opacity=0.95
    ))
    return ann

# ========== 3) 画动画散点图 ==========
# 确保列名匹配：X, Y, frameId, playerId, team
df["team"] = df["team"].astype(str)

fig = px.scatter(
    df,
    x="X",
    y="Y",
    color="team",
    animation_frame="frameId",
    animation_group="playerId",
    range_x=[0, 120],
    range_y=[0, 53.3],
    labels={"X": "", "Y": ""},
)

fig.update_layout(
    title="Video Tracking - NFL Field View",
    shapes=field_shapes_matplotlib_style(),
    annotations=field_annotations_matplotlib_style(),
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        scaleanchor="y",  # 保持 x/y 比例
        scaleratio=1
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()


   frameId          X          Y playerId  team
0        1  55.696387  46.117546        1  away
1        1  65.962253  32.052952        2  away
2        1   9.102984  30.766688        3  home
3        1  51.142200  23.314247        4  home
4        1  60.381090  46.261978        6  away
Total rows: 7538
Unique frameId: 711
Unique playerId: 116
